In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
#import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
#import datetime
import streamlit as st
from PIL import Image

In [2]:
# import BPs
p_data = 'BP_Log.csv'
p_img = 'BP_ranges.png'

In [3]:
#!pip3 install streamlit


<img src='BP_ranges.png'>

In [24]:
df = pd.read_csv(p_data)
df = df[~df['BP1'].isna()]
df[df['Date'] >= '2023-12-01']

,Date,DoW,BP1,BP2,BP3
237,2023-12-01,Friday,123/87 - 81,115/82 - 85,126/88 - 87
238,2023-12-02,Saturday,119/83 - 77,116/83 - 79,119/81 - 79
239,2023-12-03,Sunday,135/84 - 79,136/83 - 87,133/81 - 83
240,2023-12-04,Monday,125/81 - 93,125/81 - 92,132/87 - 84
241,2023-12-05,Tuesday,131/82 - 62,125/86 - 65,124/82 - 69
242,2023-12-06,Wednesday,117/81 - 69,119/82 - 71,133/86 - 69
243,2023-12-07,Thursday,124/83 - 67,128/78 - 65,138/84 - 70
244,2023-12-08,Friday,141/84 - 67,138/84 - 66,124/80 - 75
245,2023-12-09,Saturday,137/79 - 74,109/87 - 78,120/80 - 74
246,2023-12-10,Sunday,117/79 - 74,122/78 - 74,122/85 - 68


In [18]:
df['sys1'] = pd.DataFrame(df['BP1'].str.split(
    '/').to_list(), columns=['sys', 'delme'])['sys'].astype('float64')
df['sys2'] = pd.DataFrame(df['BP2'].str.split(
    '/').to_list(), columns=['sys', 'delme'])['sys'].astype('float64')
df['sys3'] = pd.DataFrame(df['BP3'].str.split(
    '/').to_list(), columns=['sys', 'delme'])['sys'].astype('float64')
df['mn_sys'] = (df['sys1'] + df['sys2'] + df['sys3']) / 3

dia1tmp = pd.DataFrame(df['BP1'].str.split(
    '/').to_list(), columns=['sys', 'delme'])['delme'].str.split('-').to_list()
df['di1'] = pd.DataFrame(dia1tmp)[0].astype('float64')
dia2tmp = pd.DataFrame(df['BP2'].str.split(
    '/').to_list(), columns=['sys', 'delme'])['delme'].str.split('-').to_list()
df['di2'] = pd.DataFrame(dia2tmp)[0].astype('float64')
dia3tmp = pd.DataFrame(df['BP3'].str.split(
    '/').to_list(), columns=['sys', 'delme'])['delme'].str.split('-').to_list()
df['di3'] = pd.DataFrame(dia3tmp)[0].astype('float64')

df['mn_di'] = (df['di1'] + df['di2'] + df['di3']) / 3


df['mn_cat'] = np.where((df['mn_sys'] > 180) | (df['mn_di'] > 120), 'Crisis',
                        np.where((df['mn_sys'] >= 140) | (df['mn_di'] >= 90), 'Stage 2',
                        np.where((df['mn_sys'] >= 130) | (df['mn_di'] >= 80), 'Stage 1',
                                 np.where((df['mn_sys'] >= 120) & (df['mn_di'] < 80), 'Elevated',
                                 np.where((df['mn_sys'] < 120) & (df['mn_di'] < 80), 'Normal', 'ERROR')))))

df['BP1_cat'] = np.where((df['sys1'] > 180) | (df['di1'] > 120), 'Crisis',
                         np.where((df['sys1'] >= 140) | (df['di1'] >= 90), 'Stage 2',
                                  np.where((df['sys1'] >= 130) | (df['di1'] >= 80), 'Stage 1',
                                           np.where((df['sys1'] >= 120) & (df['di1'] < 80), 'Elevated',
                                                    np.where((df['sys1'] < 120) & (df['di1'] < 80), 'Normal', 'ERROR')))))

df['BP2_cat'] = np.where((df['sys2'] > 180) | (df['di2'] > 120), 'Crisis',
                         np.where((df['sys2'] >= 140) | (df['di2'] >= 90), 'Stage 2',
                                  np.where((df['sys2'] >= 130) | (df['di2'] >= 80), 'Stage 1',
                                           np.where((df['sys2'] >= 120) & (df['di2'] < 80), 'Elevated',
                                                    np.where((df['sys2'] < 120) & (df['di2'] < 80), 'Normal', 'ERROR')))))

df['BP3_cat'] = np.where((df['sys3'] > 180) | (df['di3'] > 120), 'Crisis',
                         np.where((df['sys3'] >= 140) | (df['di3'] >= 90), 'Stage 2',
                                  np.where((df['sys3'] >= 130) | (df['di3'] >= 80), 'Stage 1',
                                           np.where((df['sys3'] >= 120) & (df['di3'] < 80), 'Elevated',
                                                    np.where((df['sys3'] < 120) & (df['di3'] < 80), 'Normal', 'ERROR')))))


df['mn_cdn'] = np.where((df['mn_sys'] >= 135) | (df['mn_di'] >= 85), 'High Risk',
                        np.where((df['mn_sys'] >= 121) | (df['mn_di'] > 80), 'Medium Risk', 'Normal'))

df['BP1_cdn'] = np.where((df['sys1'] >= 135) | (df['di1'] >= 85), 'High Risk',
                         np.where((df['sys1'] >= 121) | (df['di1'] > 80), 'Medium Risk', 'Normal'))

df['BP2_cdn'] = np.where((df['sys2'] >= 135) | (df['di2'] >= 85), 'High Risk',
                         np.where((df['sys2'] >= 121) | (df['di2'] > 80), 'Medium Risk', 'Normal'))

df['BP3_cdn'] = np.where((df['sys3'] >= 135) | (df['di3'] >= 85), 'High Risk',
                         np.where((df['sys3'] >= 121) | (df['di3'] > 80), 'Medium Risk', 'Normal'))


# df = df[['Date', 'DoW', 'sys1', 'di1', 'sys2', 'di2', 'sys3', 'di3', 'mn_sys', 'mn_di', 'mn_cat', 'BP1_cat', 'BP2_cat', 'BP3_cat','mn_cdn', 'BP1_cdn', 'BP2_cdn', 'BP3_cdn']]


s1 = df[['Date', 'sys1', 'di1', 'BP1_cat']]
s2 = df[['Date', 'sys2', 'di2', 'BP2_cat']]
s3 = df[['Date', 'sys3', 'di3', 'BP3_cat']]

s1.columns = ['Date', 'Systolic', 'Diastolic', 'Category']
s2.columns = ['Date', 'Systolic', 'Diastolic', 'Category']
s3.columns = ['Date', 'Systolic', 'Diastolic', 'Category']

bp_series = pd.concat([s1, s2, s3], axis=0)
# bp_series.columns = ['Date', 'Systolic', 'Diastolic', 'Category']


lst_date = pd.to_datetime(bp_series['Date'].max())
thirty_days_ago = lst_date - pd.to_timedelta(30, unit='d')

thirty_day = bp_series[pd.to_datetime(
    bp_series['Date']) > (pd.to_datetime(thirty_days_ago))]
thirty_med_sys = thirty_day['Systolic'].median()
thirty_med_di = thirty_day['Diastolic'].median()
thirty_avg_sys = thirty_day['Systolic'].mean()
thirty_avg_di = thirty_day['Diastolic'].mean()


prior_thirty = lst_date - pd.to_timedelta(30, unit='d')
prior_sixty = lst_date - pd.to_timedelta(60, unit='d')
prior = bp_series[(pd.to_datetime(bp_series['Date']) > prior_sixty) & (
    pd.to_datetime(bp_series['Date']) <= prior_thirty)]

prior_med_sys = prior['Systolic'].median()
prior_med_di = prior['Diastolic'].median()
prior_avg_sys = prior['Systolic'].mean()
prior_avg_di = prior['Diastolic'].mean()

In [19]:
print('Last 30 Day Median\nSystolic:', round(thirty_med_sys, 1),'\nDiastolic:', round(thirty_med_di, 1))

Last 30 Day Median
Systolic: 129.5 
Diastolic: 86.0


In [20]:
print('Last 30 Day Mean\nSystolic:', round(thirty_avg_sys,1),'\nDiastolic:', round(thirty_avg_di,1))

Last 30 Day Mean
Systolic: 130.1 
Diastolic: 85.0


In [21]:
print('Prior 30 Day Median\nSystolic:', round(prior_med_sys, 1),'\nDiastolic:', round(prior_med_di, 1))

Prior 30 Day Median
Systolic: 129.0 
Diastolic: 87.0


In [22]:
print('Prior 30 Day Mean\nDiastolic:', round(prior_avg_sys, 1),'\nDiastolic:', round(prior_avg_di, 1))

Prior 30 Day Mean
Diastolic: 129.3 
Diastolic: 87.0


In [23]:
df

,Date,DoW,BP1,BP2,BP3,sys1,sys2,sys3,mn_sys,di1,...,di3,mn_di,mn_cat,BP1_cat,BP2_cat,BP3_cat,mn_cdn,BP1_cdn,BP2_cdn,BP3_cdn
0,2023-04-08,Saturday,121/84 - 66,124/86 - 62,127/84 - 63,121.0,124.0,127.0,124.000000,84.0,...,84.0,84.666667,Stage 1,Stage 1,Stage 1,Stage 1,Medium Risk,Medium Risk,High Risk,Medium Risk
1,2023-04-09,Sunday,123/80 - 58,131/86 - 58,126/85 - 56,123.0,131.0,126.0,126.666667,80.0,...,85.0,83.666667,Stage 1,Stage 1,Stage 1,Stage 1,Medium Risk,Medium Risk,High Risk,High Risk
2,2023-04-10,Monday,133/85 - 59,127/85 - 59,130/88 - 59,133.0,127.0,130.0,130.000000,85.0,...,88.0,86.000000,Stage 1,Stage 1,Stage 1,Stage 1,High Risk,High Risk,High Risk,High Risk
3,2023-04-11,Tuesday,130/88 - 67,127/85 - 64,132/86 - 65,130.0,127.0,132.0,129.666667,88.0,...,86.0,86.333333,Stage 1,Stage 1,Stage 1,Stage 1,High Risk,High Risk,High Risk,High Risk
4,2023-04-12,Wednesday,122/81 - 69,122/82 - 68,127/86 - 64,122.0,122.0,127.0,123.666667,81.0,...,86.0,83.000000,Stage 1,Stage 1,Stage 1,Stage 1,Medium Risk,Medium Risk,Medium Risk,High Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,2023-12-18,Monday,133/84 - 65,132/84 - 62,127/82 - 63,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ERROR,ERROR,ERROR,ERROR,Normal,Normal,Normal,Normal
255,2023-12-19,Tuesday,151/91 - 59,139/90 - 60,136/91 - 61,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ERROR,ERROR,ERROR,ERROR,Normal,Normal,Normal,Normal
256,2023-12-20,Wednesday,126/88 - 87,115/82 - 85,126/88 - 87,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ERROR,ERROR,ERROR,ERROR,Normal,Normal,Normal,Normal
257,2023-12-21,Thursday,126/77 - 77,126/74 - 76,119/75 - 75,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ERROR,ERROR,ERROR,ERROR,Normal,Normal,Normal,Normal
